In [2]:
import pandas as pd
import json
import numpy as np

In [3]:
with open('2022-05-08.json', 'r') as file:
    data_array = []
    while True:
        s = file.readline()
        if s == "":
            break
        j = json.loads(s)
        data_array.append(j)
    
#with open('2022-05-06-true.json', 'w') as fout:
#    fout.write(json.dumps(data_array))
    
#df = pd.read_json('2022-05-06-true.json')
df = pd.DataFrame(data_array)
df.head(7)

,EVENT_NO_TRIP,EVENT_NO_STOP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,VELOCITY,DIRECTION,RADIO_QUALITY,GPS_LONGITUDE,GPS_LATITUDE,GPS_SATELLITES,GPS_HDOP,SCHEDULE_DEVIATION
0,170055937,170055942,11-OCT-20,2267,77,32955,,,,-122.602998,45.637818,12,0.7,
1,170055937,170055942,11-OCT-20,2267,104,32960,5,259,,-122.603332,45.637775,12,0.8,
2,170055937,170055942,11-OCT-20,2267,133,32965,5,267,,-122.603705,45.637763,12,0.7,
3,170055937,170055942,11-OCT-20,2267,155,32970,4,278,,-122.603998,45.637792,12,0.8,
4,170055937,170055942,11-OCT-20,2267,169,32975,2,285,,-122.604185,45.637827,12,0.7,
5,170055937,170055942,11-OCT-20,2267,173,32980,0,272,,-122.604245,45.637828,12,0.7,
6,170055937,170055942,11-OCT-20,2267,183,32985,2,270,,-122.604367,45.637828,12,0.7,


In [4]:
# data validations
def validate(df):
    try:
        assertion0 = "EVENT_NO_TRIP field is a 9 digit integer for all records."
        df['EVENT_NO_TRIP'] = df['EVENT_NO_TRIP'].astype(int)
        assert df['EVENT_NO_TRIP'].between(100000000,999999999).all(), f"{assertion0} FAILED"
    except AssertionError as warning:
        print(warning)
    else:
        print(f"{assertion0} PASSED")
        
    try:
        assertion1 = "EVENT_NO_STOP field is a 9 digit integer for all records."
        df['EVENT_NO_STOP'] = df['EVENT_NO_TRIP'].astype(int)
        assert df['EVENT_NO_STOP'].between(100000000,999999999).all(), f"{assertion1} FAILED"
    except AssertionError as warning:
        print(warning)
    else:
        print(f"{assertion1} PASSED")
        
    try:
        assertion2 = "OPD_DATE field is a date in 2020 for all records."
        assert df['OPD_DATE'].str.match('.*-20$').all() == True, f"{assertion2} FAILED"
    except AssertionError as warning:
        print(warning)
    else:
        print(f"{assertion2} PASSED")
        
    try:    
        assertion3 = "VEHICLE_ID field is a 4 digit integer for all records."
        df['VEHICLE_ID'] = df['VEHICLE_ID'].astype(int)
        assert df['VEHICLE_ID'].between(1000,9999).all(), f"{assertion3} FAILED"
    except AssertionError as warning:
        print(warning)
    else:
        print(f"{assertion3} PASSED")
        
    try:    
        assertion4 = "ACT_TIME field is in seconds between midnight up to at most 2am the next day."
        df['ACT_TIME'] = df['ACT_TIME'].astype(int)
        seconds_in_day = (60 * 60 * 24) + (60 * 60 * 2)
        assert df['ACT_TIME'].between(0,seconds_in_day).all(), f"{assertion4} FAILED"
    except AssertionError as warning:
        print(warning)
    else:
        print(f"{assertion4} PASSED")
        
    try:    
        assertion5 = "VELOCITY field exists and is greater than or equal to 0 mph for all records."
        df['VELOCITY'] = df['VELOCITY'].replace("", "0")
        df['VELOCITY'] = df['VELOCITY'].astype(float)
        assert df['VELOCITY'].all() >= 0, f"{assertion5} FAILED"
    except AssertionError as warning:
        print(warning)
    else:
        print(f"{assertion5} PASSED")
        
    try:    
        assertion6 = "The OPD_DATE field is in the format DD-MMM-YY."
        assert df['OPD_DATE'].str.match('\d\d-[A-Z]{3}-20').all(), f"{assertion6} FAILED"
    except AssertionError as warning:
        print(warning)
    else:
        print(f"{assertion6} PASSED")
        
    try:    
        assertion7 = "RADIO_QUALITY field is empty for all records."
        assert df['RADIO_QUALITY'].all() == False, f"{assertion7} FAILED"
    except AssertionError as warning:
        print(warning)
    else:
        print(f"{assertion7} PASSED")
        
    try:    
        assertion8 = "DIRECTION field will be an integer within the range [0,360)."
        df['DIRECTION'] = df['DIRECTION'].replace("", "0")
        df['DIRECTION'] = df['DIRECTION'].astype(int)
        assert df['DIRECTION'].between(0,360,inclusive="left").all(), f"{assertion8} FAILED"
    except AssertionError as warning:
        print(warning)
    else:
        print(f"{assertion8} PASSED")
        
    try:    
        assertion9 = "GPS_HDOP field is a positive floating point number."
        df['GPS_HDOP'] = df['GPS_HDOP'].replace("", "0")
        assert df['GPS_HDOP'].all() > 0, f"{assertion9} FAILED"
    except AssertionError as warning:
        print(warning)
    else:
        print(f"{assertion9} PASSED")
        
validate(df)

EVENT_NO_TRIP field is a 9 digit integer for all records. PASSED
EVENT_NO_STOP field is a 9 digit integer for all records. PASSED
OPD_DATE field is a date in 2020 for all records. PASSED
VEHICLE_ID field is a 4 digit integer for all records. PASSED
ACT_TIME field is in seconds between midnight up to at most 2am the next day. PASSED
VELOCITY field exists and is greater than or equal to 0 mph for all records. PASSED
The OPD_DATE field is in the format DD-MMM-YY. PASSED
RADIO_QUALITY field is empty for all records. PASSED
DIRECTION field will be an integer within the range [0,360). PASSED
GPS_HDOP field is a positive floating point number. PASSED


In [5]:
# data transformations
def transform(df):
    df['OPD_DATE'] = pd.to_datetime(df['OPD_DATE'])
    df['ACT_TIME'] = pd.to_timedelta(df['ACT_TIME'], unit='S', errors='ignore')
    df['OPD_DATE'] = df['OPD_DATE'] + df['ACT_TIME']

    df.rename(columns={'OPD_DATE': 'tstamp'}, inplace=True)
    df.rename(columns={'GPS_LATITUDE': 'latitude'}, inplace=True)
    df.rename(columns={'GPS_LONGITUDE': 'longitude'}, inplace=True)
    df.rename(columns={'DIRECTION': 'direction'}, inplace=True)
    df.rename(columns={'VELOCITY': 'speed'}, inplace=True)
    df.rename(columns={'EVENT_NO_TRIP': 'trip_id'}, inplace=True)
    df.rename(columns={'VEHICLE_ID': 'vehicle_id'}, inplace=True)

    df = df.drop(columns=['EVENT_NO_STOP', 'METERS', 'ACT_TIME', 'GPS_SATELLITES', 'GPS_HDOP', 'RADIO_QUALITY', 'SCHEDULE_DEVIATION'])
    return df

df = transform(df)
df

,trip_id,tstamp,vehicle_id,speed,direction,longitude,latitude
0,170055937,2020-10-11 09:09:15,2267,0.0,0,-122.602998,45.637818
1,170055937,2020-10-11 09:09:20,2267,5.0,259,-122.603332,45.637775
2,170055937,2020-10-11 09:09:25,2267,5.0,267,-122.603705,45.637763
3,170055937,2020-10-11 09:09:30,2267,4.0,278,-122.603998,45.637792
4,170055937,2020-10-11 09:09:35,2267,2.0,285,-122.604185,45.637827
...,...,...,...,...,...,...,...
135360,170055829,2020-10-11 20:57:57,6010,13.0,169,-122.601208,45.646998
135361,170055829,2020-10-11 20:58:02,6010,14.0,168,-122.601022,45.646362
135362,170055829,2020-10-11 20:58:07,6010,13.0,171,-122.600883,45.645768
135363,170055829,2020-10-11 20:58:12,6010,9.0,176,-122.60084,45.645335


In [6]:
df['service_key'] = df['tstamp'].dt.dayofweek.apply(lambda x: 'Weekday' if x <= 4 else ('Saturday' if x == 5 else 'Sunday'))
df

,trip_id,tstamp,vehicle_id,speed,direction,longitude,latitude,service_key
0,170055937,2020-10-11 09:09:15,2267,0.0,0,-122.602998,45.637818,Sunday
1,170055937,2020-10-11 09:09:20,2267,5.0,259,-122.603332,45.637775,Sunday
2,170055937,2020-10-11 09:09:25,2267,5.0,267,-122.603705,45.637763,Sunday
3,170055937,2020-10-11 09:09:30,2267,4.0,278,-122.603998,45.637792,Sunday
4,170055937,2020-10-11 09:09:35,2267,2.0,285,-122.604185,45.637827,Sunday
...,...,...,...,...,...,...,...,...
135360,170055829,2020-10-11 20:57:57,6010,13.0,169,-122.601208,45.646998,Sunday
135361,170055829,2020-10-11 20:58:02,6010,14.0,168,-122.601022,45.646362,Sunday
135362,170055829,2020-10-11 20:58:07,6010,13.0,171,-122.600883,45.645768,Sunday
135363,170055829,2020-10-11 20:58:12,6010,9.0,176,-122.60084,45.645335,Sunday


In [7]:
### Testing code
sample = []
sample.append({"name": "travis", "color": "blue", "hair": "brown"})
sample.append({"name": "mina", "color": "green", "hair": "black"})
df2 = pd.DataFrame(sample)
#df2

for x in df2:
    pass
    #print(f"This is {x}") 
#output below:
#This is name
#This is color
#This is hair

for index, row in df2.iterrows():
    pass
    #print(row['name'], row['color'], row['hair']) 
#output below:
#travis blue brown
#mina green black

In [13]:
print(df.columns)
print(len(df['trip_id']))

Index(['trip_id', 'tstamp', 'vehicle_id', 'speed', 'direction', 'longitude',
       'latitude', 'service_key'],
      dtype='object')
135365
